# A Simple Implementation of Regularized Linear Regression
This notebook goes over a implementation of Regularized Linear Regression in Python. The purpose of this notebook is to mainly break down how the linear regression algorithm/model works both through a bare bones implementation and visualization.

First, let's define the key formulas we will need to implement for our model to function:
### The Hypothesis Function
This function will actually be used to predict our outputs ($y$) value based on a given set of inputs ($x$). In this case, our $h(x)$ is going to be a linear function of $n$ dimensional inputs; $\{x_{1}, ..., x_{n}\}$, along with $\{\theta_{0}, ..., \theta_{n}\}$ trainable parameters:

$h_{\theta}(x)  =  \theta_{0} + \theta_{1} * x_{1} + ... + \theta_{n} * x_{n}  =  \theta_{0} + \sum \limits _{i=1} ^{n} \theta_{i}x_{i}$

### The Cost Function
This function will be used to assess how well our model can predict the output based on the input $x$ for each labelled data point $\{x, y\}$. We will be using the mean squared error to calculate the cost of each prediction. The $m$ value is the number of training points. Both $x^{(i)}$ and $\theta$ are vectors of size $n\times1$. The second term is the regularization term. When the cost function is being optimized, it will aide in stopping the model from overfitting the training parameters onto the training set. $\lambda$ will control how much weight we want to give the regularization term, this means if $\lambda$ is set too large, the model might have a high bias and perform poorly. Note, $\theta_{0}$ is not regularized.

$J(\theta)  =  \frac{1}{2m}  \sum \limits _{i=1} ^{m} (h_{\theta}(x^{(i)}) - y^{(i)})^{2} + \frac{\lambda}{2m}\sum \limits _{j=1} ^{n} \theta_{j}^{2}$

### The Gradient Calculation Function
The optimization of the cost function will be done with gradient descent. For this, the gradients of all the trainable parameters must be calculated, that is, the partial derivitives of $\{\frac{\partial J(\theta)}{\partial \theta_{0}}, ..., \frac{\partial J(\theta)}{\partial \theta_{n}}\}$. After calculating the partiel derivatives, the result should be:

$\frac{\partial J(\theta)}{\partial \theta_{0}}  =  \frac{1}{m}  \sum \limits _{i=1} ^{m} {(h_{\theta}(x^{(i)}) - y^{(i)})}$

$\frac{\partial J(\theta)}{\partial \theta_{0}}  =  \frac{1}{m}  \sum \limits _{i=1} ^{m} {(h_{\theta}(x^{(i)}) - y^{(i)}) x_{j} ^{i}  +  \frac{\lambda}{m} \theta_{j}}   \hspace{10mm} \text{for} \hspace{3mm}j = 1, ..., n$

With these calulations, the gradient descent algorithm can be used to optimize the cost function and train the model. Here is a implementation of Linear Regression.

In [147]:
import numpy as np

class RegularizedLinearRegression:
    def __init__(self, n=1, reg_lambda=0.01, learning_rate=0.001, epsilon=1e-9):
        self.n = n
        self.theta = np.zeros((n + 1, 1))
        self.grad = np.zeros((n + 1, 1))
        self.reg_lambda = reg_lambda
        self.learning_rate = learning_rate
        self.epsilon
        
    def train(self, X, y, iterations=None):
        
        # get number of training examples and insert m by 1 bias vector
        m = X.shape[0]
        X = np.insert(X, [0], np.ones((m, 1)), axis=1)
        
        cost_hist = np.zeros(iterations)
        
        for i in range(iterations):
            
            # run batch gradient descent step and calculate cost of updated theta
            self._gradient_descent_batch_step(X, y, m)
            cost_hist[i] = self._cost_func(X, y)
            
        return cost_hist
            
    def _gradient_descent_batch_step(self, X, y, m):

        # calculate gradients of theta (calculated grad of theta_0 again to remove the regularization calculation from the vectoried cal.)
        self.grad = (1/m) * np.transpose(np.matmul(np.transpose(np.sum(np.subtract(self._hypothesis_func(X), y), axis=1)), X)) + (self.reg_lambda/m) * self.theta 
        self.grad[0] = (1/m) * np.sum(np.subtract(self._hypothesis_func(X), y))
        
        # update theta
        self.theta = self.theta - self.learning_rate * self.grad
        
    def _hypothesis_func(self, X):
        # hypothesis function for prediction
        return np.matmul(X, self.theta)
    
    def _cost_func(self, X, y):
        # mean squared error cost
        m = X.shape[0]
        return 1/(2 * m) * (np.sum(np.power(np.subtract(self._hypothesis_func(X), y), 2))  +  self.reg_lambda * np.sum(np.power(self.theta[1:], 2)))